In [5]:
#ejecutar en la terminal export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import datetime

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from utils import label_map_util
from utils import visualization_utils as vis_util

import tensorflow as tf

print(tf.version) ##solo funciona para la version 1.* de tensorflow no para la v2

<module 'tensorflow._api.v1.version' from '/anaconda3/lib/python3.6/site-packages/tensorflow/_api/v1/version/__init__.py'>


In [6]:
##CONSTANTES 

NUM_CLASSES = 1 ## solo nos interesa detectar la clase persona

intervaloCapturaMin = 10 ## minima cantidad de segundos que tiene que pasar entre capturas
tiempoCaptura = 0 ##guarda cuando se realizo la ultima captura en segundos
umbralCaptura = 0.75 ##minimo umbral para realizar captura (en porcentaje)

#inputVideo = cv2.VideoCapture(0) ##entrada por web cam
inputVideo = cv2.VideoCapture("testVideos/vuelo_retiro.mp4") ##entrada de video (para depuracion solo)

In [12]:
##MODELOS
# no hace falta que esten descargados, se descargan en el momento
# link: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md
#MODEL_NAME = 'ssd_inception_v2_coco_2017_11_17'
MODEL_NAME = 'ssd_mobilenet_v1_coco_2018_01_28'


In [13]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = 'modelos/' + MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('labels', 'label.pbtxt')



# Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


# Loading label map
# Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


# Helper code
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)


# Detection
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        while True:

            # Read frame from camera
            ret, image_np = inputVideo.read()
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Extract image tensor
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Extract detection boxes
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Extract detection scores
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            # Extract detection classes
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            # Extract number of detectionsd
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            
            vis_util.visualize_boxes_and_labels_on_image_array(
                
                
                
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),

                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)

            # Display output
            cv2.imshow('object detection', cv2.resize(image_np, (800, 600)))
            
            #####
            
            
            if(scores[0][0] >= umbralCaptura): 
                if (datetime.datetime.now().second + datetime.datetime.now().minute*60 - tiempoCaptura >= intervaloCapturaMin):
                    print("captura")
                    coords = (45.562565, -11.467381) #las sacaremos del modulo gps
                    tiempoCaptura = datetime.datetime.now().second + datetime.datetime.now().minute*60
                    cv2.putText(image_np, str(coords) + " " + str(datetime.datetime.now()) , 
                           (10,700), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 55, 255), 2, cv2.LINE_AA)
                    cv2.imwrite("capturas/" + str(datetime.datetime.now()) +'.png', image_np)
            
            #####
            

            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

captura


KeyboardInterrupt: 